In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Churn_Customer_Lifetime_Value_prediction/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Churn_Customer_Lifetime_Value_prediction'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_file: Path
    model_1: Path
    model_1_scaler: Path
    model_2: Path
    model_1_stats: Path
    model_2_stats: Path

In [4]:
from churn_pred.constants import *
from churn_pred.utils.main_utils import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        
        create_directories([config.root_dir, config.model_1_stats, config.model_2_stats])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_file=Path(config.test_file),
            model_1=Path(config.model_1),
            model_1_scaler=Path(config.model_1_scaler),
            model_2=Path(config.model_2),
            model_1_stats=Path(config.model_1_stats),
            model_2_stats=Path(config.model_2_stats)
        )
        
        return model_evaluation_config

In [5]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from churn_pred.utils.main_utils import save_json, plot_confusion_matrix

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def data_prepatarion(self):
        test_data = pd.read_csv(self.config.test_file)
        test_data = test_data.replace([np.inf, -np.inf], np.nan)
        
        lg_y_test = test_data['churn_next_month']
        xgb_y_test = test_data['early_churn']
        
        X_test = test_data.drop(columns=['churn_next_month', 'early_churn'])
        
        return X_test, lg_y_test, xgb_y_test
    
    def evaluate_lg_model(self, X_test, y_test):
        with open(self.config.model_1, 'rb') as file:
            model = pickle.load(file)
        
        with open(self.config.model_1_scaler, 'rb') as file:
            scaler = pickle.load(file)
        
        X_test = scaler.transform(X_test)
        
        y_pred_proba = model.predict_proba(X_test)[:,1]
        y_pred = model.predict(X_test)

        # Evaluate
        auc = {}
        auc['auc'] = roc_auc_score(y_test, y_pred_proba)
        save_json(self.config.model_1_stats, auc, 'logistic_regression_auc')
        
        report = classification_report(y_test, y_pred, output_dict=True)
        save_json(self.config.model_1_stats, report, 'logistic_regression_report')
        
        cm = confusion_matrix(y_test, y_pred)
        plot_confusion_matrix(cm, self.config.model_1_stats, 'logistic_regression')
        
        # print(auc, report)
    
    def evaluate_xgb_model(self, X_test, y_test):
        with open(self.config.model_2, 'rb') as file:
            model = pickle.load(file)
        
        y_pred_proba = model.predict_proba(X_test)[:,1]
        y_pred = model.predict(X_test)

        # Evaluate
        auc = {}
        auc['auc'] = roc_auc_score(y_test, y_pred_proba)
        save_json(self.config.model_2_stats, auc, 'xgb_classifier_auc')
        
        report = classification_report(y_test, y_pred, output_dict=True)
        save_json(self.config.model_2_stats, report, 'xgb_classifier_report')
        
        cm = confusion_matrix(y_test, y_pred)
        plot_confusion_matrix(cm, self.config.model_2_stats, 'xgb_classifier')
        
        # print(report)
    
    def evaluation(self):
        X_test, lg_y_test, xgb_y_test = self.data_prepatarion()
        
        self.evaluate_lg_model(X_test, lg_y_test)
        
        self.evaluate_xgb_model(X_test, xgb_y_test)

In [6]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluation()

except Exception as e:
    raise e

[2025-06-24 16:09:00,593: INFO: main_utils: yaml file: config/config.yaml loaded successfully]
[2025-06-24 16:09:00,601: INFO: main_utils: yaml file: params.yaml loaded successfully]
[2025-06-24 16:09:00,605: INFO: main_utils: yaml file: schema.yaml loaded successfully]
[2025-06-24 16:09:00,607: INFO: main_utils: created directory at: artifacts]
[2025-06-24 16:09:00,610: INFO: main_utils: created directory at: artifacts/model_evaluation]
[2025-06-24 16:09:00,613: INFO: main_utils: created directory at: artifacts/model_evaluation/model_1_stats]
[2025-06-24 16:09:00,615: INFO: main_utils: created directory at: artifacts/model_evaluation/model_2_stats]
[2025-06-24 16:09:00,757: INFO: main_utils: json file saved at: artifacts/model_evaluation/model_1_stats/logistic_regression_auc.json]
[2025-06-24 16:09:00,778: INFO: main_utils: json file saved at: artifacts/model_evaluation/model_1_stats/logistic_regression_report.json]
[2025-06-24 16:09:01,285: INFO: main_utils: json file saved at: artif